# Demo: How to Modify Models Using GRADIEND

1. Select a model to modify

In [1]:
from transformers import AutoModel, AutoTokenizer, pipeline, logging

# Suppress warnings from Hugging Face transformers library
logging.set_verbosity_error()

In [3]:
import yaml

model = 'bert-base-cased'
distilbert_de = 'distilbert-base-german-cased'
bert_de = 'bert-base-german-cased'

config = yaml.safe_load(open("config.yml"))['F_N']

2. Train the GRADIEND model

In [ ]:
from gradiend.training.gradiend_training import train

# you may override some default behavior of gradiend.training.trainer.train() with the model_config
model_config = {
    'eval_max_size': 0.5, # use all of the validation data
    'epochs': 1,
}

gradiend_model_dir = train(distilbert_de, model_config, config, n=1)

3. [Optional]: Analyze the Encoder

In [ ]:
from gradiend.evaluation.analyze_encoder import analyze_models
from gradiend.export.encoder_stats import print_encoder_stats
from gradiend.export.encoder_plot import plot

analyze_models(gradiend_model_dir)
print_encoder_stats(model)

# plot the encoded values distribution across different datasets
plot(gradiend_model_dir)

4. Analyze the Decoder

In [ ]:
from gradiend.evaluation.analyze_decoder import default_evaluation
default_evaluation(gradiend_model_dir)

5. Create modified models based on the base models by selecting parameters based on the analysis and the BPI, FPI, and MPI metrics

In [ ]:
from gradiend.evaluation.select_models import select
result = select(gradiend_model_dir, force=False, plot=False)

6. Load the modified models and do something with them

In [ ]:
for suffix in ['N', 'F', 'M']:
    model_name = f'results/changed_models/{model}-{suffix}'
    print(f'Loading model {model_name}')
    modified_model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # do something with the model
    # ...
    
    # Example: Use the pipeline to predict the masked word    
    fill_mask = pipeline("fill-mask", model=model_name, tokenizer=model_name)
    text = 'The man worked as a [MASK].'
    result = fill_mask(text)
    predicted = result[0]['token_str']
    predicted_prob = result[0]['score']
    print(f'Predicted for {suffix}: {predicted} ({predicted_prob})')

7. [Optional]: Evaluate the modified models on a simple masking task to evaluate overfitting

In [ ]:
from gradiend.evaluation.analyze_decoder import evaluate_gender_prediction_for_models
from gradiend.export.gender_predictions import plot_all

for targets in [('man', 'woman'), ('woman', 'man')]:
    evaluate_gender_prediction_for_models(model, target_words=targets)
    suffix = '_'.join(targets)
    plot_all(f'results/gender_prediction/{model}.csv', suffix=suffix)

8. [Optional]: Generate some example predictions for the model

In [ ]:
from gradiend.export.example_predictions import run_for_model
run_for_model(model)